In [1]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets.mnist import load_data

In [2]:
def define_discriminator(input_shape = (28, 28, 1)):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(20, (5, 5), padding = 'same', input_shape = input_shape))
    model.add(keras.layers.LeakyReLU(alpha = 0.1))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv2D(20, (5, 5), padding = 'same', input_shape = input_shape))    # Change input size to input shape 
    model.add(keras.layers.LeakyReLU(alpha = 0.1))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv2D(20, (5, 5), padding = 'same', input_shape = input_shape))
    model.add(keras.layers.LeakyReLU(alpha = 0.1))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv2D(20, (5, 5), padding = 'same', input_shape = input_shape))
    model.add(keras.layers.LeakyReLU(alpha = 0.1))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [3]:
def define_generator(latent_dim):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(150, input_shape = (latent_dim,)))
    model.add(keras.layers.LeakyReLU(alpha = 0.1))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(300, input_shape = (latent_dim,)))
    model.add(keras.layers.LeakyReLU(alpha = 0.1))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(450, input_shape = (latent_dim,)))
    model.add(keras.layers.LeakyReLU(alpha = 0.1))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(600, input_shape = (latent_dim,)))    #convert latent_dim to an array (latent_dim,)
    model.add(keras.layers.LeakyReLU(alpha = 0.1))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(784, input_shape = (latent_dim,)))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Reshape((28, 28, 1)))
    # model.add(keras.activations.sigmoid())
    model.add(keras.layers.Activation('sigmoid'))  #change this here , use .layers as well, above gives errors 
    
    return model

In [4]:
def define_gans(g_model, d_model):
    d_model.trainable = False
    model = keras.Sequential()
    model.add(g_model)
    model.add(d_model)
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [5]:
def load_real_samples():
    (train_x,_),(_,_)=load_data()
    train_x=np.expand_dims(train_x,axis=-1)
    train_x=train_x.astype('float32')
    train_x=train_x/255

    return train_x

In [6]:
def generate_real_samples(dataset,n_samples):
    ids=np.random.randint(0,dataset.shape[0],n_samples)
    dataset=dataset[ids]
    y=np.ones((n_samples,1))
    return dataset,y

In [7]:
def generate_latent_noise(latent_dim,n_samples):
    noise=np.random.randn(latent_dim*n_samples)
    noise=noise.reshape(n_samples,latent_dim)
    return noise

In [8]:
def generate_fake_samples(g_model,latent_dim,n_samples):
    noise=generate_latent_noise(latent_dim,n_samples)
    fake_data=g_model.predict(noise)
    y=np.zeros((n_samples,1))
    return fake_data, y

In [9]:
def summary(g_model,d_model,dataset,latent_dim,epoch,n_samples=30):
    x_real,y_real=generate_real_samples(dataset,n_samples)
    loss_real,acc_real=d_model.evaluate(x_real,y_real)       #spelling error for evaluate
    x_fake,y_fake=generate_fake_samples(g_model,latent_dim,n_samples)
    loss_fake,acc_fake=d_model.evaluate(x_fake,y_fake)
    print(f'Accuracy real:{acc_real*100}%, Accuracy fake:{acc_fake*100}%')

In [10]:
def train(g_model,d_model,define_gan,dataset,latent_dim,n_epoch=10,n_batch=500):
    batches_per_epoch=int(dataset.shape[0]/n_batch)
    n_samples=250
    for i in range(n_epoch):
        for j in range(batches_per_epoch):
            x_real,y_real=generate_real_samples(dataset,n_samples)
            x_fake,y_fake=generate_fake_samples(g_model,latent_dim,n_samples)

            d_loss_real,_=d_model.train_on_batch(x_real,y_real)
            d_loss_fake,_=d_model.train_on_batch(x_fake,y_fake)

            d_loss=d_loss_fake+d_loss_real

            noise=generate_latent_noise(latent_dim,n_samples)

            y_gan=np.ones((n_samples,1))   # replace -1 with 1
            g_loss,_=define_gan.train_on_batch(noise,y_gan)
            print(f'on epoch:{i+1}, Disc Loss: {d_loss}, Gen Loss: {g_loss}')

            if i % 10 ==0:
                summary(g_model,d_model,dataset,latent_dim,i)
        

In [ ]:
latent_dim=100
d_model=define_discriminator()
g_model=define_generator(latent_dim)
gan=define_gans(g_model, d_model)
dataset=load_real_samples()
train(g_model,d_model,gan,dataset,latent_dim)